<a href="https://colab.research.google.com/github/GreatLearningAIML1/gurgaon-feb-batch-abhishek02git/blob/master/CV_Project2_Dog_Breed_Classification_Questions_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 2.x

TensorFlow 2.x selected.


In [0]:
#CV_Project2_Dog_Breed_Classification_Questions-1.ipynb
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import r2_score, mean_squared_error
from scipy.stats import zscore

import tensorflow as tf
import keras
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import MaxPooling2D
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.models import Sequential
from keras.constraints import maxnorm
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.utils import np_utils
from keras.optimizers import SGD
from tqdm import tqdm
import os
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array
from keras.preprocessing.image import load_img
from keras.preprocessing.image import ImageDataGenerator
import argparse
import imutils
import cv2
import time

import h5py

Using TensorFlow backend.


## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

### Load Dataset Files

In [0]:
!pip install kaggle

     |████████████████████████████████| 122kB 2.8MB/s 
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.8.2 which is incompatible.
ERROR: google-colab 1.0.0 has requirement requests~=2.21.0, but you'll have requests 2.22.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement six~=1.12.0, but you'll have six 1.13.0 which is incompatible.
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
  Found existing installation: urllib3 1.25.7
    Uninstalling urllib3-1.25.7:
      Successfully uninstalled urllib3-1.25.7


In [0]:
from google.colab import files
uploaded = files.upload()

In [0]:
for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!ls ~/.kaggle

User uploaded file "kaggle.json" with length 67 bytes
kaggle.json


In [0]:
!kaggle competitions download -c dog-breed-identification

  0% 0.00/214k [00:00<?, ?B/s]
100% 214k/214k [00:00<00:00, 86.1MB/s]
  0% 0.00/281k [00:00<?, ?B/s]
100% 281k/281k [00:00<00:00, 39.9MB/s]
 97% 337M/346M [00:07<00:00, 46.5MB/s]
100% 346M/346M [00:07<00:00, 51.2MB/s]
100% 343M/345M [00:06<00:00, 41.4MB/s]
100% 345M/345M [00:06<00:00, 53.3MB/s]


Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [0]:
project_path = '/content/'

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'train.zip', 'r') as z:
  z.extractall()

Repeat the same step for test.zip

In [0]:
#Already done
!unzip -q test.zip

Repeat the same step for sample_submission.csv.zip

In [0]:
#already done separately
!unzip -q sample_submission.csv.zip

Repeat the same step for labels.csv.zip

In [0]:
#Already done above
!unzip -q labels.csv.zip

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [0]:
train = pd.read_csv(project_path + 'labels.csv')
train.head(5)

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


In [0]:
test = pd.read_csv(project_path + 'sample_submission.csv')
test.head(1)

,id,affenpinscher,afghan_hound,african_hunting_dog,airedale,american_staffordshire_terrier,appenzeller,australian_terrier,basenji,basset,beagle,bedlington_terrier,bernese_mountain_dog,black-and-tan_coonhound,blenheim_spaniel,bloodhound,bluetick,border_collie,border_terrier,borzoi,boston_bull,bouvier_des_flandres,boxer,brabancon_griffon,briard,brittany_spaniel,bull_mastiff,cairn,cardigan,chesapeake_bay_retriever,chihuahua,chow,clumber,cocker_spaniel,collie,curly-coated_retriever,dandie_dinmont,dhole,dingo,doberman,...,norwegian_elkhound,norwich_terrier,old_english_sheepdog,otterhound,papillon,pekinese,pembroke,pomeranian,pug,redbone,rhodesian_ridgeback,rottweiler,saint_bernard,saluki,samoyed,schipperke,scotch_terrier,scottish_deerhound,sealyham_terrier,shetland_sheepdog,shih-tzu,siberian_husky,silky_terrier,soft-coated_wheaten_terrier,staffordshire_bullterrier,standard_poodle,standard_schnauzer,sussex_spaniel,tibetan_mastiff,tibetan_terrier,toy_poodle,toy_terrier,vizsla,walker_hound,weimaraner,welsh_springer_spaniel,west_highland_white_terrier,whippet,wire-haired_fox_terrier,yorkshire_terrier
0,000621fb3cbb32d8935728e48679680e,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,...,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333,0.008333


### Print the count of each category of Dogs given in the dataset



In [0]:
train.breed.value_counts()

scottish_deerhound                126
maltese_dog                       117
afghan_hound                      116
entlebucher                       115
bernese_mountain_dog              114
shih-tzu                          112
pomeranian                        111
great_pyrenees                    111
basenji                           110
samoyed                           109
tibetan_terrier                   107
airedale                          107
cairn                             106
leonberg                          106
beagle                            105
japanese_spaniel                  105
blenheim_spaniel                  102
australian_terrier                102
miniature_pinscher                102
irish_wolfhound                   101
lakeland_terrier                   99
saluki                             99
papillon                           96
norwegian_elkhound                 95
whippet                            95
siberian_husky                     95
pug         

In [0]:
# there is some class imbalance in the databset. the breed with highest frequency has twice the images than breed with lowest frequency

### Get one-hot encodings of labels

In [0]:
for_dummy = train['breed']
y_encoded = pd.get_dummies(for_dummy, prefix="breed")

In [0]:
y_encoded.head(1)

,breed_affenpinscher,breed_afghan_hound,breed_african_hunting_dog,breed_airedale,breed_american_staffordshire_terrier,breed_appenzeller,breed_australian_terrier,breed_basenji,breed_basset,breed_beagle,breed_bedlington_terrier,breed_bernese_mountain_dog,breed_black-and-tan_coonhound,breed_blenheim_spaniel,breed_bloodhound,breed_bluetick,breed_border_collie,breed_border_terrier,breed_borzoi,breed_boston_bull,breed_bouvier_des_flandres,breed_boxer,breed_brabancon_griffon,breed_briard,breed_brittany_spaniel,breed_bull_mastiff,breed_cairn,breed_cardigan,breed_chesapeake_bay_retriever,breed_chihuahua,breed_chow,breed_clumber,breed_cocker_spaniel,breed_collie,breed_curly-coated_retriever,breed_dandie_dinmont,breed_dhole,breed_dingo,breed_doberman,breed_english_foxhound,...,breed_norwegian_elkhound,breed_norwich_terrier,breed_old_english_sheepdog,breed_otterhound,breed_papillon,breed_pekinese,breed_pembroke,breed_pomeranian,breed_pug,breed_redbone,breed_rhodesian_ridgeback,breed_rottweiler,breed_saint_bernard,breed_saluki,breed_samoyed,breed_schipperke,breed_scotch_terrier,breed_scottish_deerhound,breed_sealyham_terrier,breed_shetland_sheepdog,breed_shih-tzu,breed_siberian_husky,breed_silky_terrier,breed_soft-coated_wheaten_terrier,breed_staffordshire_bullterrier,breed_standard_poodle,breed_standard_schnauzer,breed_sussex_spaniel,breed_tibetan_mastiff,breed_tibetan_terrier,breed_toy_poodle,breed_toy_terrier,breed_vizsla,breed_walker_hound,breed_weimaraner,breed_welsh_springer_spaniel,breed_west_highland_white_terrier,breed_whippet,breed_wire-haired_fox_terrier,breed_yorkshire_terrier
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [0]:
from tqdm import tqdm
import cv2
import pickle

In [0]:
X_features = []
X_features = []
y_features = []
img_rows = 70
img_cols = 70
num_channels = 3

In [0]:
#create the object and create a pickle dat fiel for this.
pickle_X_features = project_path + "X_features.dat"
pickle_X_features_gray = project_path + "X_features_gray.dat"

In [0]:
# function to read images as arrays
def read_image(img_path, mode = 'color', resize = False, size = 224):
    '''
    Default mode is : color(BGR) --> color(RGB)
    Other modes allowed are : 'grayscale' and 'include_opacity'
    '''
    if mode == 'grayscale':
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    elif mode == 'include_opacity':
        img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    else:
        img = cv2.imread(img_path, cv2.IMREAD_COLOR)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    if resize == True:
        img = cv2.resize(img, (size, size))
    
    return img

In [0]:
for (f, img) in tqdm(train[['id', 'breed']].values):
  train_img = cv2.imread(project_path + 'train/{0}.jpg'.format(f))
  train_img_resize = cv2.resize(train_img, (img_rows, img_cols))
  X_features.append(train_img_resize)

100%|██████████| 10222/10222 [00:20<00:00, 501.75it/s]


In [0]:
y_features = y_encoded.values

In [0]:
# Its important to use binary mode 
dbfile = open(pickle_X_features, 'ab')
pickle.dump(X_features, dbfile)

In [0]:
np.asarray(y_features).shape

(10222, 120)

Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [0]:
X_features = np.asarray(X_features)/255.0

In [0]:
X_features.shape

(10222, 70, 70, 3)

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X_features, y_features, test_size=0.3, random_state = 1)

In [0]:
#trainY = tf.keras.utils.to_categorical(y_train, num_classes=120)
#testY = tf.keras.utils.to_categorical(y_test, num_classes=120)

### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [0]:
test_img = test[['id']]

In [0]:
test_img.values[0][0]

'000621fb3cbb32d8935728e48679680e'

Run the below code to load the test image files in x_test_feature

In [0]:
import glob

In [0]:
X_test_features = []
for img in glob.glob("/content/test/*.jpg"):
    X_test_features.append(read_image(img, resize = True, size = img_rows))


Normalize the test data and convert it into 4 dimensions

In [0]:
X_test_features = np.asarray(X_test_features)/255.0

### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [0]:
num_classes = 120

In [0]:
model = Sequential()
model.add(Conv2D(256, (3, 3), input_shape=(img_rows, img_cols, 3), activation='relu'))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

### Use batch_size = 128 and epochs = 10 and execute the model

In [0]:
batch_size = 128
epochs = 10

In [0]:
# Compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_2 (Conv2D)            (None, 68, 68, 256)       7168      
_________________________________________________________________
flatten_2 (Flatten)          (None, 1183744)           0         
_________________________________________________________________
dense_2 (Dense)              (None, 120)               142049400 
Total params: 142,056,568
Trainable params: 142,056,568
Non-trainable params: 0
_________________________________________________________________
None


In [0]:
from keras.callbacks import ModelCheckpoint
mc = ModelCheckpoint('best_model.h5', monitor='val_acc', mode='max', verbose=1, save_best_only=True)

(3067, 70, 70, 3)

In [0]:
# Fit the model
start = time.clock() 
# Training the model
model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test,y_test), callbacks=[mc])
end = time.clock()

print("Train Time: {} ".format(end-start)) 

Train on 7155 samples, validate on 3067 samples
Epoch 1/10
7155/7155 [==============================] - 32s 4ms/step - loss: 5.2343 - acc: 0.0172 - val_loss: 4.6896 - val_acc: 0.0222

Epoch 00001: val_acc improved from -inf to 0.02217, saving model to best_model.h5
Epoch 2/10
7155/7155 [==============================] - 29s 4ms/step - loss: 2.4683 - acc: 0.5117 - val_loss: 5.3352 - val_acc: 0.0473

Epoch 00002: val_acc improved from 0.02217 to 0.04728, saving model to best_model.h5
Epoch 3/10
7155/7155 [==============================] - 29s 4ms/step - loss: 0.2481 - acc: 0.9627 - val_loss: 6.7770 - val_acc: 0.0483

Epoch 00003: val_acc improved from 0.04728 to 0.04826, saving model to best_model.h5
Epoch 4/10
7155/7155 [==============================] - 29s 4ms/step - loss: 0.0321 - acc: 0.9972 - val_loss: 7.6320 - val_acc: 0.0473

Epoch 00004: val_acc did not improve from 0.04826
Epoch 5/10
7155/7155 [==============================] - 29s 4ms/step - loss: 0.0104 - acc: 0.9996 - val_lo

#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [0]:
train_df = pd.read_csv(project_path + 'labels.csv')
train_df['file'] = train_df['id'] + '.jpg'

test_df = pd.read_csv(project_path + 'sample_submission.csv')
test_df['file'] = test_df['id'] + '.jpg'

In [0]:
from tensorflow.python.keras.applications.resnet50 import preprocess_input
from tensorflow.python.keras.preprocessing.image import ImageDataGenerator

image_size = 224
datagen = ImageDataGenerator(preprocessing_function=preprocess_input, 
                             rescale=1./255.,
                             horizontal_flip=True,
                             width_shift_range = 0.2,
                             height_shift_range = 0.2,
                             validation_split=0.2)

In [0]:
train_location = project_path + "train/"
test_location = project_path + "test/"

### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [0]:
train_generator=datagen.flow_from_dataframe(
                        dataframe=train_df,
                        directory= train_location,
                        x_col="file",
                        y_col="breed",
                        has_ext=False,
                        subset="training",
                        batch_size=32,
                        seed=50,
                        shuffle=True,
                        class_mode="categorical",
                        target_size=(img_rows, img_cols))

Found 8178 validated image filenames belonging to 120 classes.


In [0]:
val_generator=datagen.flow_from_dataframe(
                        dataframe = train_df,
                        directory = train_location,
                        x_col="file",
                        y_col="breed",
                        has_ext=False,
                        subset="validation",
                        batch_size=1,
                        seed=42,
                        shuffle=True,
                        class_mode="categorical",
                        target_size=(img_rows, img_cols))

Found 2044 validated image filenames belonging to 120 classes.


In [0]:
test_datagen=ImageDataGenerator(preprocessing_function=preprocess_input,rescale=1./255.)

test_generator=test_datagen.flow_from_dataframe(
                            dataframe=test_df,
                            directory=test_location,
                            x_col="file",
                            y_col=None,
                            has_ext=False,
                            batch_size=1,
                            seed=42,
                            shuffle=False,
                            class_mode=None,
                            target_size=(img_rows, img_cols))

Found 10357 validated image filenames.


In [0]:
train_gen = image.ImageDataGenerator(rotation_range=15, width_shift_range=0.1, shear_range=0.25,
                               height_shift_range=0.1, zoom_range=0.2, horizontal_flip = True)
val_gen = image.ImageDataGenerator() # No need for data augmentation on Validation Data.

In [0]:
train_batches = train_gen.flow(X_train, y_train, batch_size = 128)
val_batches = val_gen.flow(X_test, y_test, batch_size = 128)

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [0]:
model_aug = Sequential([
        BatchNormalization(input_shape = (70,70,3)),
        Convolution2D(128,(5,5), activation='relu'),
        BatchNormalization(),
        Convolution2D(128,(3,3), activation='relu'),
        BatchNormalization(),
        MaxPooling2D(),
        Flatten(),
        Dropout(0.3),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(120, activation='softmax')
        ])
model_aug.compile(optimizer = 'adam' , loss='categorical_crossentropy', metrics=['accuracy'])








Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.



In [0]:
# Fit the model
start = time.clock() 
#model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size)
model_aug.fit_generator(train_batches, (X_train.shape[0]//train_batches.batch_size), epochs=10,
                    validation_data = val_batches, validation_steps = (X_test.shape[0]//val_batches.batch_size))
end = time.clock()

print("Train Time: {} ".format(end-start)) 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/10
55/55 [==============================] - 20s 372ms/step - loss: 11.0684 - acc: 0.0116 - val_loss: 4.8817 - val_acc: 0.0092
Epoch 2/10
55/55 [==============================] - 12s 218ms/step - loss: 4.8729 - acc: 0.0117 - val_loss: 4.7895 - val_acc: 0.0102
Epoch 3/10
55/55 [==============================] - 11s 203ms/step - loss: 4.7883 - acc: 0.0110 - val_loss: 4.7861 - val_acc: 0.0085
Epoch 4/10
55/55 [==============================] - 11s 205ms/step - loss: 4.7992 - acc: 0.0111 - val_loss: 4.8106 - val_acc: 0.0105
Epoch 5/10
55/55 [==============================] - 11s 205ms/step - loss: 4.7948 - acc: 0.0115 - val_loss: 4.7852 - val_acc: 0.0085
Epoch 6/10
55/55 [==============================] - 11s 206ms/step - loss: 4.7839 - acc: 0.0112 - val_loss: 4.8346 - val_acc: 0.0143
Epoch 7/10
55/55 [==============================] - 12s 209ms/step - loss: 4.7906 - acc: 0.0098 - val_loss:

# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

Use the below code to load VGG16 weights trained on ImageNet

In [0]:
from keras.applications.vgg16 import VGG16

In [0]:
num_classes = 120

In [0]:
from keras.layers import Activation

In [0]:
from keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(weights=('/content/drive/My Drive/Colab Notebooks/gl_aiml/r8_lab/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5'), include_top=False, pooling='avg')

Print the summary of the base_model

In [0]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         (None, None, None, 3)     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, None, None, 64)    1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, None, None, 64)    36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, None, None, 64)    0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, None, None, 128)   73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, None, None, 128)   147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, None, None, 128)   0     

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

In [0]:
model = Sequential()
model.add(base_model)
model.add(Dense(1024))
model.add(Activation('relu'))
model.add(Dropout(0.2))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.3))
model.add(Dense(num_classes, activation='softmax'))

### Make all the layers in the base_model (VGG16) to be non-trainable

In [0]:
model.layers[0].trainable = False

### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%

In [0]:
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
STEP_SIZE_TRAIN=train_generator.n
STEP_SIZE_VALID=val_generator.n

print(STEP_SIZE_TRAIN)
print(STEP_SIZE_VALID)

model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=val_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

8178
2044
Epoch 1/10
8178/8178 [==============================] - 968s 118ms/step - loss: 4.4821 - acc: 0.0499 - val_loss: 4.1480 - val_acc: 0.0837
Epoch 2/10
8178/8178 [==============================] - 973s 119ms/step - loss: 3.9439 - acc: 0.1122 - val_loss: 3.9205 - val_acc: 0.1027
Epoch 3/10
8178/8178 [==============================] - 966s 118ms/step - loss: 3.7177 - acc: 0.1449 - val_loss: 3.8892 - val_acc: 0.1179
Epoch 4/10
8178/8178 [==============================] - 965s 118ms/step - loss: 3.5706 - acc: 0.1687 - val_loss: 3.8145 - val_acc: 0.1272
Epoch 5/10
8178/8178 [==============================] - 968s 118ms/step - loss: 3.4570 - acc: 0.1896 - val_loss: 3.7830 - val_acc: 0.1419
Epoch 6/10
8178/8178 [==============================] - 966s 118ms/step - loss: 3.3524 - acc: 0.2049 - val_loss: 3.7979 - val_acc: 0.1360
Epoch 7/10
8178/8178 [==============================] - 965s 118ms/step - loss: 3.2567 - acc: 0.2238 - val_loss: 3.7661 - val_acc: 0.1502
Epoch 8/10
8178/8178 [==